# Регрессия - пора творить!

Если вы дошли до этой практики и выполнили все предыдущие - вау! Аплодисменты вам!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/leo.png" width=500/></p>



Тем не менее, не время расслабляться! На этот раз мы не будем рассказывать вам новый материал - лишь дадим пару напутствий и пожелаем успехов и сил!

В этот раз вам предлагается разработать модель предсказания цены домов по набору данных Boston Houses Pricing. Мы его уже немного пощупали в прошлой практике, но в этой вам предстоит сделать всё самостоятельно и на полных данных!

Что надо будет сделать:
- загрузить данные;
- посмотреть и размер, и примеры данных;
- проверить количество пропусков в данных;
    - убедиться, что пропусков в данных нет и можно строить **baseline**;
- разделить данные на train и test;
    - Делаем это один раз! Все дальнейшие модификации и улучшения должны делаться на основе единственного разделения!
    - И не забудьте зафиксировать сид! Переменную `RANDOM_SEED` мы даём ниже;
- создать baseline модель, получить метрики на тестовой выборке;
- проанализировать данные, подготовить план по очистке и предобработке данных;
- создать функцию очистки;
- произвести очистку train и test выборок -> обучаем модель с очищенными данными;
    - Снимаем метрики и так мы оцениваем влияние очистки данных на обучение;
- создать класс предобработки данных;
- сделать предобработку очищенных данных -> обучаем модель;
    - Снимаем метрики и так мы оцениваем влияние предобработки данных на обучение;

А дальше можете пофантазировать, почитать интернет и посмотреть, как улучшить метрики! 

**Ваша цель - достигнуть наилучших показателей!**

Вот вам пара идей по интересной организации предобработке:
- по признаку `CRIM` создать признак `is_CRIM`, который будет равен 1, если `CRIM` не равен 0 и 0, если `CRIM` равен нулю;
- сделать обучение модели [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), посмотреть, какие признаки имеют веса близкие к 0 и убрать их из обучения как незначимые;
- оставить только признаки с высокой корреляцией с целевой переменной;
- посмотреть, какие переменные коррелирцют между собой и оставить только единственные из пар коррелирующих;
- посмотреть, в каких примерах данных модель ошибается сильнее всего (отсортировать записи из теста по величине ошибки) и проанализировать, есть ли какая-то зависимость или шум;
- комбинировать разные подходы в зависимости от того, помогает это предсказаниям или нет.

Ещё большим плюсом будет, если вы покажете, как влияет каждый этап предобработки на результаты обучения!

In [1]:
import pandas as pd
df_src = pd.read_csv("boston.csv")

In [2]:
# Зафиксируем состояние случайных чисел
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

print (df_src.head())
print (df_src.info())

print("Количество пропусков в данных: ", df_src.isnull().sum())

if df_src.isnull().sum().sum() == 0:
    print("Пропусков в данных нет. Приступайте к построению baseline")
else:
    print("В данных присутствуют пропуски. Следует очистить их.")

target_column = 'MEDV'
x = df_src.drop(target_column, axis=1)
y = df_src[target_column]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)

baseline_model = LinearRegression()
baseline_model.fit(x_train, y_train)

baseline_predictions = baseline_model.predict(x_test)
mse = mean_squared_error(y_test, baseline_predictions)
r2 = r2_score(y_test, baseline_predictions)
print(f"Результаты работы первой итерации. baseline MSE: {mse:.3f}, r2: {r2:.3f}")

def clean_data(df):
    df['is_CRIM'] = (df['CRIM'] > 0).astype(int)
    return df

x_train_clean = clean_data(x_train.copy())
x_test_clean = clean_data(x_test.copy())

model_clean = LinearRegression()
model_clean.fit(x_train_clean, y_train)
predictions_clean = model_clean.predict(x_test_clean)

mse_clean = mean_squared_error(y_test, predictions_clean)
r2_clean = r2_score(y_test, predictions_clean)
print(f"Результаты итерации по очистки данных. Cleaned MSE: {mse_clean:.3f}, R2: {r2_clean:.3f}")

class DataPreprocessor:
    def __init__(self):
        pass
    
    def transform(self, df):
        df['is_CRIM'] = (df['CRIM'] > 0).astype(int)
        return df

preprocessor = DataPreprocessor()
x_train_preprocessed = preprocessor.transform(x_train_clean.copy())
x_test_preprocessed = preprocessor.transform(x_test_clean.copy())

model_preprocessed = LinearRegression()
model_preprocessed.fit(x_train_preprocessed, y_train)
predictions_preprocessed = model_preprocessed.predict(x_test_preprocessed)

mse_preprocessed = mean_squared_error(y_test, predictions_preprocessed)
r2_preprocessed = r2_score(y_test, predictions_preprocessed)

print("Окончательная итерация. Результаты итерации будет ниже.")
print(f"Preprocessed MSE: {mse_preprocessed:.3f}, R2: {r2_preprocessed:.3f}")


      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-nu

Что мы ждем от вас? 

Готовый ноутбук с таким же названием, в котором будут обязательные пункты с выводами по анализу данных и анализу поведения модели, а также дополнительные пункты с вашими мыслями и результатами улучшения модели.

Успехов вам, мы в вас верим, вы точно справитесь!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/cat-computer-peek-out-36960208.jpg" width=600/></p>